<a href="https://colab.research.google.com/github/secutron/RunTime/blob/master/G1_DeepFaceLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clone or pull DeepFaceLab from Github

Mode = "clone" #@param ["clone", "pull"]

from pathlib import Path
if (Mode == "clone"):
  !git clone https://github.com/iperov/DeepFaceLab.git --branch DFL-1.0
  print("Repository cloned!")
else:
  %cd /content/DeepFaceLab
  !git pull
  print("Repository pulled!")

!pip install -r /content/DeepFaceLab/requirements-colab.txt
!pip install --upgrade scikit-image
!apt-get install cuda-10-0
print("Requirements installed!")
if not Path("/content/pretrain").exists():
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/pretrain-CelebA/pretrain_CelebA.zip' -O pretrain_CelebA.zip
  !mkdir /content/pretrain
  !unzip -q /content/pretrain_CelebA.zip -d /content/pretrain/
  !rm /content/pretrain_CelebA.zip
  print("Pretrained model downloaded!")

In [ ]:
#@title Import Data
import os
Mode = "data_dst" #@param ["data_src", "data_dst"]
File_name = "iron.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+File_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+File_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $File_name
  name = mode + '.mp4'
  old_name = File_name.split(".")[0] + '.mp4'
  !mv $old_name $name

def create(path,mode):
  data_src = "workspace/data_src"
  data_dst = "workspace/data_dst"
  data_src_aligned = "workspace/data_src/aligned"
  data_dst_aligned = "workspace/data_dst/aligned"
  %cd $path
  !mkdir $mode
  !mkdir $data_src
  !mkdir $data_dst
  !mkdir $data_src_aligned
  !mkdir $data_dst_aligned


# if Mode == "workspace":
if not os.path.exists("/content/workspace"):
  create("/content", "workspace")
if Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
print("Done!")

In [ ]:
#@title Extract frames of video
Video = "data_dst" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"
  
!python $cmd

In [ ]:
#@title Denoise frames
Data = "data_dst" #@param ["data_src", "data_dst"]
Factor = 1 #@param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/"+Data+" --factor "+str(Factor)

%cd "/content"
!python $cmd

In [ ]:
#@title Detect faces
Data = "data_dst" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "S3FD (Mark only)", "S3FD (Extract unaligned)", "MT"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
facetype = None

if Detector == "S3FD":
  detect_type = "s3fd"
elif Detector == "S3FD (Mark only)":
  detect_type = "s3fd"
  facetype = "mark_only"
elif Detector == "S3FD (Extract unaligned)":
  detect_type = "s3fd"
  facetype = "full_face_no_align"
elif Detector == "MT":
  detect_type = "mt"

folder = "workspace/"+Data
folder_align = folder+"/aligned"
debug_folder = folder_align+"/debug"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_align

if Debug:
  cmd+= " --debug-dir "+debug_folder

cmd+=" --detector "+detect_type
  
if facetype:
  cmd+=" --face-type "+facetype
  
%cd "/content"
!python $cmd

In [ ]:
#@title Align faces
Data = "data_dst" #@param ["data_src", "data_dst"]
sort_type = "absdiff" #@param ["absdiff","hist", "hist-dissim", "face-yaw", "face-pitch", "vggface", "blur", "final", "final-no-blur"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

In [ ]:
#@title Enhance faceset data
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/workspace/"+Data+"/aligned"

cmd = "/content/DeepFaceLab/main.py facesettool enhance --input-dir "+data_path
!python $cmd

In [ ]:
#@title Training
Model = "SAEHD" #@param ["SAEHD", "SAE", "Quick96", "H128", "LIAEF128", "DF", "AVATAR", "DEV_FANSEG"]
Backup_every_hour = True #@param {type:"boolean"}

%cd "/content"

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (Backup_every_hour):
  if not os.path.exists('workspace.zip'):
    print("Creating workspace archive ...")
    !zip -r -q workspace.zip workspace
    print("Archive created!")
  else:
    print("Archive exist!")

if (Backup_every_hour):
  print("Time to end session: "+str(round((43200-uptime)/3600))+" hours")
  backup_time = str(3600)
  backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace/model'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
elif (round(39600-uptime) > 0):
  print("Time to backup: "+str(round((39600-uptime)/3600))+" hours")
  backup_time = str(round(39600-uptime))
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""
    
cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model
  
if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

In [ ]:
#@title Convert video
Model = "SAEHD" #@param ["SAEHD", "SAE", "Quick96", "H128", "LIAEF128", "DF", "AVATAR" ]

if Model == "AVATAR":  
  cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst/aligned --output-dir workspace/data_dst/merged --model-dir workspace/model --model "+Model
else:
  cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [ ]:
#@title Get result video and copy to Drive 

!python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4
!cp /content/workspace/result.mp4 /content/drive/My\ Drive/